In [80]:
import numpy as np

In [96]:
def Build_Tree(file_name):
    file = open(file_name)

    tree = {}

    for line in file:
        dist = line[4:5]
        if dist == 'N':
            mu = int(line[6:7])
            sigma = int(line[8:9])
            tree[(line[:1],line[2:3])] = [(dist,mu,sigma),lambda t:np.exp(mu*t)*np.exp((1/2)*(sigma**2)*(t**2)),0]
        if dist == 'E':
            lam = int(line[6:7])
            tree[(line[:1],line[2:3])] = [(dist,lam),lambda t: lam/(lam-t),0]
        if dist == 'U':
            a = int(line[6:7])
            b = int(line[8:9])
            tree[(line[:1],line[2:3])] = [(dist,a,b),lambda t: (np.exp(t*b)-np.exp(t*a))/(t*(b-a)),0]
        if dist == 'P':
            lam = int(line[6:7])
            tree[(line[:1],line[2:3])] = [(dist,lam),lambda t: np.exp(lam*(np.exp(t)-1)),0]
    return tree

In [151]:
def Simulate(value):
    dist = value[0][0]
    if dist == 'N':
        value[2] = np.random.normal(value[0][1],value[0][2])
    if dist == 'E':
        value[2] = np.random.exponential(value[0][1])
    if dist == 'U':
        value[2] = np.random.uniform(value[0][1],value[0][2])
    if dist == 'P':
        value[2] = np.random.poisson(value[0][1])

def Simulate_Edges(tree):
    for keys in tree:
        Simulate(tree[keys])

Tree = Build_Tree("test.txt")
Simulate_Edges(Tree)

In [150]:
def Connection_Tree(tree):
    connection_tree = {}
    for keys in tree:
        if keys[0] not in connection_tree:
            connection_tree[keys[0]] = set(keys[1])
        if keys[1] not in connection_tree:
            connection_tree[keys[1]] = set(keys[0])
        if keys[0] in connection_tree:
            connection_tree[keys[0]].add(keys[1])
        if keys[1] in connection_tree:
            connection_tree[keys[1]].add(keys[0])
    return connection_tree

def DFS(connection_tree, source, observer: chr):
    stack = [(source, [source])]
    visited = set()
    while stack:
        (vertex, path) = stack.pop()
        if vertex not in visited:
            if vertex == observer:
                return path
            visited.add(vertex)
            for neighbor in connection_tree[vertex]:
                stack.append((neighbor,path + [neighbor]))

def Path_Edge(path):
    edges = []
    for i in range(len(path)-1):
        edges.append((path[i],path[i+1]))
    return edges

def Infection_Simulation(tree, observers, source):
    infection_times = []
    connection_tree = Connection_Tree(tree)
    for observer in observers:
        path = DFS(connection_tree, source, observer)
        edges = Path_Edge(path)
        time = 0
        for key in edges:
            if key in Tree:
                time += Tree[key][2]
            else:
                flipped_key = (key[1],key[0])
                time += Tree[flipped_key][2]
        infection_times.append(time)
    return infection_times

[1.233021804323024, 0]
